In [1]:
import warnings; warnings.filterwarnings("ignore")
import os;
from sys import platform
print(os.getcwd())
if platform == "darwin":
    os.chdir('/Users/ziyingc/Dropbox/Lab/MotorCortex')
elif platform == "win32":
    os.chdir('D:\Dropbox\Lab\MotorCortex')
print(os.getcwd())
import numpy as np
import comparison as cmp
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.patches import Patch as patch
from matplotlib.gridspec import GridSpec
import matplotlib.cm as cm; 
import matplotlib.mlab as mlab
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pycwt as wavelet; mother = wavelet.Morlet(6)
from scipy import stats
import read_data as rd;   import time2frequency as t2f
import behavior_track as bt ; import behavior_plot as bp ; import turning_track  as tt
import philters
fps = 30; fs_out = 1000; fs = 30000; T = 1/fs_out;
N = int((1/fps)/T/2+1);

sgn   = ['***','**','*']
P_sgn = np.array([0.0001, 0.001, 0.05])

/Users/ziyingc/Dropbox/Lab/MotorCortex/Multi_Rats_ver2.0
/Users/ziyingc/Dropbox/Lab/MotorCortex


In [3]:
# erfdn = np.load('Rats/Rats_LT_ephy.npy')
# erfdn = np.load('Rats/Rats_Turn_ephy.npy')
erfdn = np.load('Rats/Rats_ephy.npy')
mode = 'FFT'
f = np.load('Rats/freqs_'+mode+'_'+str(fs_out)+'.npy');x_fit = np.log10(f); 


idx_f1_fit = np.logical_and(f>=12.50, f<=15.00);
idx_f2_fit = np.logical_and(f>=48.75, f<=57.50); idx_f_fit = np.logical_or(idx_f1_fit, idx_f2_fit);
for rfdn in erfdn:
    
    epn    = np.load(rfdn+'tracking'+'/epn_ephy.npy')
    subset = np.load(rfdn+'tracking'+'/subset_ephy_li.npy');
    print(rfdn[9:len(rfdn)-1], len(epn))
    slope_l, intercept_l, slope_i, intercept_i, intercept_li = \
    t2f.flicker_fit_fft(rfdn, epn, fs_out, subset, f, idx_f_fit, mode)
    slope     = np.array([slope_l,     slope_i]);
    intercept = np.array([intercept_l, intercept_i]);

    np.save(rfdn+'tracking'+'/slope_epn_ephy.npy',    slope)
    np.save(rfdn+'tracking'+'/intercept_epn_ephy.npy',intercept)
#     np.save(rfdn+'tracking'+'/slope_epn_turn_elec_post.npy',    slope)
#     np.save(rfdn+'tracking'+'/intercept_epn_turn_elec_post.npy',intercept)

Swashbuckler 23
Outlaw 62
Quartus 29
Unknown 16


ValueError: Cannot load file containing pickled data when allow_pickle=False

In [ ]:
erfdn = np.load('Rats/Rats_Turn_ephy.npy')
# erfdn = np.load('Rats/Rats_LT_ephy.npy')[[0,1,2,3]]
mode = 'FFT'
f = np.load('Rats/freqs_'+mode+'_'+str(fs_out)+'.npy');x_fit = np.log10(f); 
test = False
flr = np.arange(10, 30, 10); rc  = len(erfdn)
fhr = flr+1.24;

idx_band =  np.logical_and(f>=flr[0], f<=fhr[0])
# f_win = np.hanning(len(f[idx_band]));
f_win = np.hamming(len(f[idx_band]));
# f_win = np.ones((len(f[idx_band]),))
print('band frequency:',f_win)

sgn   = ['***','**','*']
P_sgn = np.array([0.0001, 0.001, 0.05])

fig = plt.figure(1, figsize=(20, rc*4))
gs  = GridSpec(rc , 1);
xtk    = np.arange(1, 4*len(flr)+1, 4)
xticklabel = [];
rr = 0
for rfdn in erfdn:
    
    epn    = np.load(rfdn+'tracking'+'/epn_turn_elec_post.npy')
    subset = np.load(rfdn+'tracking'+'/subset_ephy_li.npy');
    print(rfdn[9:len(rfdn)-1], len(epn))
    
    slope     = np.load(rfdn+'tracking'+'/slope_epn_turn_elec_post.npy')
    intercept = np.load(rfdn+'tracking'+'/intercept_epn_turn_elec_post.npy')
    
    rt_brst    = np.empty((2,len(epn),len(flr)))
    rt_brst_ac = np.zeros((2,len(epn),len(flr)))-1;
    rt_brst_ref= np.zeros((2,len(epn),len(flr)))-1;
    
    T_ac  = np.zeros((len(epn),len(flr)))-1;
    T_brst  = np.empty((2,len(epn),len(flr)), dtype=object)
    i_brst  = np.empty((2,len(epn),len(flr)), dtype=object)
    i_brst_max = np.empty((2,len(epn),len(flr)), dtype=object)

    kk = 0
    for k in epn:
        fdn = rfdn + k + '/'
        spd_f = np.load(fdn+'tracking/'+'spd_kal_f.npy');N_win = 10
        spd_f = np.convolve(spd_f, np.ones((N_win,))/N_win, mode='same')
        idx_clean_f   = np.load(fdn+'tracking/'+'idx_clean_f.npy');
        idx_clean_fft = np.load(fdn+'tracking/'+'idx_clean_fft.npy')
        idx_clean   = np.logical_and(idx_clean_f, idx_clean_fft)
        
        ts_f = np.load(fdn+'tracking/'+'ts_f.npy'); l_t = len(ts_f)

        v_thr = 5 #6 cm/s
        T_ts = len(ts_f[idx_clean])/fps/60
        idx_spd = np.logical_and(idx_clean, spd_f <= v_thr)
        
        i_ref = 1
        i = subset[i_ref]
        chn = str(i//10)+str(i%10)
        Y_f = np.load(fdn + mode+'_'+str(fs_out)+'/ch.'+chn+'.fps.npy').T
        y_fit = 10**(x_fit*slope[i_ref,kk] + intercept[i_ref,kk]);     
        Y_f_fit_ref = (Y_f.T/y_fit)

        jj = 0
        for i in subset:
            chn = str(i//10)+str(i%10)
            Y_f = np.load(fdn + mode+'_'+str(fs_out)+'/ch.'+chn+'.fps.npy').T
            y_fit = 10**(x_fit*slope[jj,kk] + intercept[jj,kk]);     
            Y_f_fit = (Y_f.T/y_fit)
            for ii in np.arange(len(flr)):
                fl = flr[ii];     fh = fhr[ii]
                idx_band =  np.logical_and(f>=fl, f<=fh)
                P_f_ref = (np.matmul(Y_f_fit_ref[:,idx_band],f_win)/np.sum(f_win))[idx_clean]
                m_f     = np.mean(P_f_ref);
                md_f    = np.median(P_f_ref);
                std_f   = np.std(P_f_ref);
                thr1 = m_f+std_f*3
                thr2 = thr1/2
                P_f = np.matmul(Y_f_fit[:,idx_band],f_win)/np.sum(f_win)

                index, T = philters.read_burst(P_f, thr2, thr1, idx_clean, 1)
                rt_brst[jj, kk, ii] = len(index)/T_ts; 
                T_brst [jj, kk, ii] = T; 
                i_brst [jj, kk, ii] = index;
                i_brts = np.empty((0,));

                for i_b in np.arange(len(index)):
                    i_brt = index[i_b];  T_brt = T[i_b]
                    i_brt_max = np.argmax(P_f[i_brt:i_brt+T_brt])
                    i_brts = np.append(i_brts, i_brt+i_brt_max)
                i_brst_max[jj, kk, ii] = i_brts
            jj = jj+1
        kk = kk+1
        


    rr = rr + 1




# Lesion